### Wrangle Exercises - Regression Module

In [1]:
#Imports for the function:
import numpy as np
import pandas as pd
from env import get_db_url
import os
import warnings
warnings.filterwarnings('ignore')


**Build the acquire function and explore the pulled dataset**

In [2]:
def acquire_zillow():
    '''
    '''
    #assign the file name
    filename = 'zillow.csv'
    #check if the file exists in the current directory and read it if it is
    if os.path.exists(filename):
        print('Reading from csv file...')
        return pd.read_csv(filename)
    #assign the sql query to a variable for use
    query = '''
    SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
    FROM properties_2017
    WHERE propertylandusetypeid = 261
    '''
    #if needed pull a fresh copy of the dataset from the database and save localy
    print('Getting a fresh copy from SQL database...')
    df = pd.read_sql(query, get_db_url('zillow'))
    df.to_csv(filename, index=False)
    return df  

In [3]:
df = acquire_zillow()
df.head()

Reading from csv file...


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [4]:
df.shape

(2152863, 7)

In [5]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [6]:
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [7]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


**Build the data cleaning function**
- many columns have substantial outliers so we need to build a function to remove outliers

In [8]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    for col in col_list:
        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [9]:
df.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [10]:
out_columns = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'taxamount']
df = remove_outliers(df, 1.5, out_columns)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
18,3.0,1.0,1244.0,169471.0,1950.0,2532.88,6037.0
19,3.0,2.0,1300.0,233266.0,1950.0,3110.99,6037.0
20,3.0,2.0,1222.0,290492.0,1951.0,3870.25,6037.0


In [11]:
df.shape


(1855285, 7)

In [12]:
df.isnull().sum()

bedroomcnt                        0
bathroomcnt                       0
calculatedfinishedsquarefeet      0
taxvaluedollarcnt                 0
yearbuilt                       874
taxamount                         0
fips                              0
dtype: int64

In [13]:
#still need to remove a few rows with null values in 'yearbuilt' column
df = df.dropna(subset =['yearbuilt'])
    

In [14]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
18,3.0,1.0,1244.0,169471.0,1950.0,2532.88,6037.0
19,3.0,2.0,1300.0,233266.0,1950.0,3110.99,6037.0
20,3.0,2.0,1222.0,290492.0,1951.0,3870.25,6037.0


In [15]:
df.shape

(1854411, 7)

In [16]:
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [17]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06
mean,3.214658e+00,2.053666e+00,1.653812e+03,3.279717e+05,1.959445e+03,4.069593e+03,6.048332e+03
std,7.814270e-01,7.076689e-01,5.645706e+02,1.989700e+05,2.054604e+01,2.242310e+03,2.052412e+01
min,2.000000e+00,1.000000e+00,1.000000e+00,2.200000e+01,1.801000e+03,1.648000e+01,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.231000e+03,1.738660e+05,1.949000e+03,2.366740e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.548000e+03,2.966380e+05,1.957000e+03,3.747360e+03,6.037000e+03
75%,4.000000e+00,2.500000e+00,1.988000e+03,4.525000e+05,1.972000e+03,5.487795e+03,6.059000e+03
max,5.000000e+00,4.000000e+00,3.410000e+03,9.509400e+05,2.016000e+03,1.037784e+04,6.111000e+03


**Cleaning notes**
- first we needed to remove the significant outliers from all the columns with the exception of the 'fips' and the 'yearbuilt' columns
- then we dropped the rows with null values in the 'yearbuilt' column because there were only 874 out of almost two million and because the range between min and max year built is so large that a median year built might skew the data or not provide valueable insight

### Use the function from the wrangle.py file to acquire and clean the data
- after moving the above created functions to the wrangle.py file and combining them into one wrangle_zillow function, **restart the kernal** and run the process using the created function from the file

In [1]:
### RESTART THE KERNEL ###

In [2]:
import numpy as np
import pandas as pd
from env import get_db_url
import os
import warnings
warnings.filterwarnings('ignore')


from wrangle import wrangle_zillow

In [3]:
df = wrangle_zillow()
df.head()

Reading from csv file...


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
18,3.0,1.0,1244.0,169471.0,1950.0,2532.88,6037.0
19,3.0,2.0,1300.0,233266.0,1950.0,3110.99,6037.0
20,3.0,2.0,1222.0,290492.0,1951.0,3870.25,6037.0


In [4]:
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [5]:
df.shape

(1854411, 7)

In [6]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06,1.854411e+06
mean,3.214658e+00,2.053666e+00,1.653812e+03,3.279717e+05,1.959445e+03,4.069593e+03,6.048332e+03
std,7.814270e-01,7.076689e-01,5.645706e+02,1.989700e+05,2.054604e+01,2.242310e+03,2.052412e+01
min,2.000000e+00,1.000000e+00,1.000000e+00,2.200000e+01,1.801000e+03,1.648000e+01,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.231000e+03,1.738660e+05,1.949000e+03,2.366740e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.548000e+03,2.966380e+05,1.957000e+03,3.747360e+03,6.037000e+03
75%,4.000000e+00,2.500000e+00,1.988000e+03,4.525000e+05,1.972000e+03,5.487795e+03,6.059000e+03
max,5.000000e+00,4.000000e+00,3.410000e+03,9.509400e+05,2.016000e+03,1.037784e+04,6.111000e+03
